In [1]:
from typing import List, TypedDict, Annotated
from pydantic import BaseModel, Field
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser , PydanticOutputParser
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph import StateGraph, START, END
from langgraph.types import Send
from pathlib import Path
import operator

In [2]:
class Task(BaseModel):
    id: int
    title: str
    brief: str = Field(..., description="What to cover")
class Plan(BaseModel):
    blog_title: str
    tasks: List[Task]
class State(TypedDict):
    topic: str
    plan: Plan
    # reducer: results from workers get concatenated automatically
    sections: Annotated[List[str], operator.add]
    final: str

In [3]:

structured_llm = ChatOllama(
    model="deepseek-r1:latest",
    format="json",
    temperature=0
)

# LLM for content generation (without JSON mode)
content_llm = ChatOllama(
    model="llama3.2:latest",
    temperature=0.7
)


### Orchestration Code

In [4]:
class State(TypedDict):
    topic: str
    plan : Plan
    sections: Annotated[List[str], operator.add]
    final: str

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
google_model = ChatGoogleGenerativeAI(model = "gemini-2.5-flash")

In [97]:
def orchestrator(state: State) -> dict:

    plan = google_model.with_structured_output(Plan).invoke(
        [
            SystemMessage(
                content=(
                    "Create a blog plan with 5-7 sections on the following topic."
                )
            ),
            HumanMessage(content=f"Topic: {state['topic']}"),
        ]
    )
    return {"plan": plan}

In [6]:
def orchestrator(state: State) -> dict:
    """Creates a blog outline with sections"""
    
    # Setup parser for Plan schema
    parser = PydanticOutputParser(pydantic_object=Plan)
    
    # Create prompt
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Create a blog plan  5with-7 sections.\n\n{format_instructions}"),
        ("human", "Topic: {topic}")
    ])
    prompt = prompt.partial(format_instructions=parser.get_format_instructions()) # tell the output rules to the prompt automatically
    
    # Build and run chain
    chain = prompt | structured_llm | parser
    plan_dict = chain.invoke({"topic": state["topic"]})
    
    # CRITICAL FIX: Convert dict to Pydantic Plan object
    # The parser returns a dict, but State expects a Plan object
    plan_object = Plan(**plan_dict)
    
    return {"plan": plan_object}

In [99]:
or_graph = StateGraph(State)
or_graph.add_node("orchestrator" , orchestrator)

or_graph.add_edge(START, "orchestrator")
or_graph.add_edge("orchestrator", END)
app_or = or_graph.compile()

In [100]:
output = app_or.invoke({"topic": "Write a blog on Self Attention"})

In [101]:
output

{'topic': 'Write a blog on Self Attention',
 'plan': Plan(blog_title='Understanding Self Attention Mechanisms', tasks=[Task(id=1, title='Introduction to Self Attention', brief="Explain what self attention is and why it's important in modern AI."), Task(id=2, title='How Self Attention Works', brief='Detail the mathematical and computational aspects of the self-attention mechanism.'), Task(id=3, title='Benefits of Self Attention', brief='Discuss the advantages over previous methods, like improved accuracy and efficiency.'), Task(id=4, title='Comparison with Traditional Methods', brief='Compare self attention with attention mechanisms or other models like RNNs.'), Task(id=5, title='Applications in AI', brief='Explore real-world uses, such as in transformers, NLP, and computer vision.'), Task(id=6, title='Challenges and Limitations', brief='Address issues like computational cost, interpretability, and ethical concerns.'), Task(id=7, title='Future Directions', brief='Speculate on upcoming d

In [102]:
for task in output["plan"].tasks:
    print({
            "task": task,
            "topic": output["topic"],
            "plan": output["plan"]
        })

{'task': Task(id=1, title='Introduction to Self Attention', brief="Explain what self attention is and why it's important in modern AI."), 'topic': 'Write a blog on Self Attention', 'plan': Plan(blog_title='Understanding Self Attention Mechanisms', tasks=[Task(id=1, title='Introduction to Self Attention', brief="Explain what self attention is and why it's important in modern AI."), Task(id=2, title='How Self Attention Works', brief='Detail the mathematical and computational aspects of the self-attention mechanism.'), Task(id=3, title='Benefits of Self Attention', brief='Discuss the advantages over previous methods, like improved accuracy and efficiency.'), Task(id=4, title='Comparison with Traditional Methods', brief='Compare self attention with attention mechanisms or other models like RNNs.'), Task(id=5, title='Applications in AI', brief='Explore real-world uses, such as in transformers, NLP, and computer vision.'), Task(id=6, title='Challenges and Limitations', brief='Address issues 

### Fan OutCode

In [103]:
def fanout(state: State):
    """Creates parallel Send commands for each task"""
    
    # CRITICAL FIX: Access plan.tasks as attribute, NOT plan["tasks"]
    # state["plan"] is a Plan object (Pydantic), not a dict
    return [
        Send("worker", {
            "task": task,
            "topic": state["topic"],
            "plan": state["plan"]
        })
        for task in state["plan"].tasks  # Use .tasks attribute
    ]

In [104]:
def worker(payload: dict) -> dict:
    """Generates markdown content for one section"""
    
    # Extract data from payload
    task = payload["task"]
    topic = payload["topic"]
    plan = payload["plan"]
    
    # CRITICAL FIX: Access plan.blog_title as attribute
    blog_title = plan.blog_title  # Use .blog_title attribute, NOT ["blog_title"]
    
    # Generate content using content_llm (without JSON mode)
    response = content_llm.invoke([
        SystemMessage(content="Write one clean Markdown section."),
        HumanMessage(content=f"""
Blog: {blog_title}
Topic: {topic}

Section: {task.title}
Brief: {task.brief}

Write 2-3 paragraphs. Return only the section content in Markdown.
""")
    ])
    
    section_md = response.content.strip()
    
    # IMPORTANT: Return as list because sections uses operator.add reducer
    # Multiple worker returns get automatically concatenated into sections list
    return {"sections": [section_md]}


In [105]:
def reducer(state: State) -> dict:
    """Combines sections into final blog and saves to file"""
    
    # CRITICAL FIX: Access attributes, not dict keys
    title = state["plan"].blog_title  # Use .blog_title attribute
    
    # Join all sections with double newlines
    body = "\n\n".join(state["sections"]).strip()
    
    # Create final markdown
    final_md = f"# {title}\n\n{body}\n"
    
    # Save to file
    filename = title.lower().replace(" ", "_") + ".md"
    Path(filename).write_text(final_md, encoding="utf-8")
    
    print(f"✅ Saved to {filename}")
    
    return {"final": final_md}

In [106]:
g = StateGraph(State)

# Add all nodes
g.add_node("orchestrator", orchestrator)
g.add_node("worker", worker)
g.add_node("reducer", reducer)

# Connect nodes
g.add_edge(START, "orchestrator")
g.add_conditional_edges("orchestrator", fanout, ["worker"])  # Parallel fanout
g.add_edge("worker", "reducer")
g.add_edge("reducer", END)

# Compile
app = g.compile()

In [107]:
result = app.invoke({
        "topic": "Self Attention in Transformers",
        "sections": []  # Must initialize as empty list
    })

✅ Saved to demystifying_self-attention_in_transformers:_an_in-depth_exploration.md


In [108]:
print(result["final"])

# Demystifying Self-Attention in Transformers: An In-Depth Exploration

# Introduction to Transformers and Attention
## Overview of Transformer Architecture

Transformers have revolutionized the field of natural language processing (NLP) with their ability to efficiently handle long-range dependencies in sequences. Introduced by Vaswani et al. in 2017, the Transformer architecture replaces traditional recurrent neural networks (RNNs) with self-attention mechanisms to model complex interactions between input elements. This shift enables models to focus on specific parts of the input sequence that are relevant for a particular task.

## Role of Attention Mechanisms

Attention mechanisms play a crucial role in Transformers by allowing them to weigh the importance of different input elements when generating output. The attention mechanism enables the model to selectively focus on certain parts of the input data, rather than relying solely on sequential dependencies. This allows Transformer